In [0]:
USE CATALOG c_bikeshop;

-- CTE para calcular o faturamento total por produto e categoria
WITH product_category_sales AS (
  SELECT
    c.category_name,
    p.product_name,
    SUM(bs.net_amount) AS product_total_revenue
  FROM
    gold.bike_sales AS bs
    JOIN silver.dim_product AS p ON bs.product_id = p.product_id
    JOIN silver.dim_categories AS c ON p.category_id = c.category_id
  GROUP BY
    c.category_name,
    p.product_name
)
-- Query final para calcular o ranking e o percentual de participação usando Window Functions
SELECT
  category_name,
  product_name,
  product_total_revenue AS product_revenue,
  -- WINDOW FUNCTION 1: Calcula o ranking de cada produto dentro de sua categoria
  RANK() OVER (
    PARTITION BY category_name
    ORDER BY
      product_total_revenue DESC
  ) AS sales_rank_in_category,
  -- WINDOW FUNCTION 2: Calcula o percentual de participação do produto no faturamento total da categoria
  (
    product_total_revenue / SUM(product_total_revenue) OVER (
      PARTITION BY category_name
    )
  ) * 100 AS participation_percentage
FROM
  product_category_sales
ORDER BY
  category_name,
  sales_rank_in_category;